# 导入包

In [69]:
import numpy as np
import pandas as pd

base = "/content/drive/MyDrive/Colab Notebooks/Statistical-Learning-Method-Python/CH04-Naive-Bayes"

# 导入数据
数据来源李航《统计学习方法》P63

In [70]:
data_path = base + "/CH04-data1.txt" 
data = np.array(pd.read_csv(data_path, header=None, names=["x1", "x2", "y"]))

cols = data.shape[1]
X = data[:, :cols-1]
y = data[:, cols-1:]
X,y

(array([[1, 'S'],
        [1, 'M'],
        [1, 'M'],
        [1, 'S'],
        [1, 'S'],
        [2, 'S'],
        [2, 'M'],
        [2, 'M'],
        [2, 'L'],
        [2, 'L'],
        [3, 'L'],
        [3, 'M'],
        [3, 'M'],
        [3, 'L'],
        [3, 'L']], dtype=object), array([[-1],
        [-1],
        [1],
        [1],
        [-1],
        [-1],
        [-1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [-1]], dtype=object))

# 朴素贝叶斯法的参数估计

## 指示函数
$$
sign(y,\hat{y})=
\left\{\begin{matrix}
1,\ \ \ y=\hat{y}\\
0,\ \ \ y\ne\hat{y}
\end{matrix}\right.
$$

In [71]:
def sign(a, b):
  return (int)(a == b)

## 极大似然估计

### 计算先验和条件概率（极大似然估计）

先验概率$P(Y=c_k)$的极大似然估计：
$$
P(Y=c_k)=\frac{\sum_{i=1}^NI(y_i=c_k)}{N}，k=1,2,\dots,K
$$

设第$j$个特征$x^{(j)}$可能取值的集合为$\{a_{j1},a_{j2},\dots,a_{jS_j}\}$

***

条件概率$P(X^{(j)}=a_{jl}|Y=c_k)$的极大似然估计：
$$
P(X^{(j)}=a_{jl}|Y=c_k)=\frac{\sum_{i=1}^NI(x_i^{(j)}=a_{jl},y_i=c_k)}{\sum_{i=1}^NI(y_i=c_k)} \\ 
j=1,2,\dots,n；l=1,2,\dots,S_j；k=1,2,\dots,K
$$

其中：$x_i^{(j)}$是第$i$个样本的第$j$个特征；$a_{jl}$是第$j$个特征可能取得第$l$个值

In [72]:
def PriorProbability(y, y_hat):
  prior = 0
  for i in range(y.shape[0]):
    prior += sign(y[i], y_hat)
  prior /= y.shape[0]

  return prior

In [73]:
def ConditionalProbability(X, y, x, y_hat):
  # X - 训练数据集的特征
  # y - 训练数据集的标签值
  # x - 待分类数据的特征
  # y_hat - 待分类数据的标签值
  
  ret = 1.0
  for j in range(x.shape[1]):
    numerator, denominator = 0, 0

    for i in range(X.shape[0]):
      denominator += sign(y[i], y_hat)
      numerator += (int)(sign(y[i], y_hat) and sign(X[i,j], x[0,j]))
    ret *= numerator / denominator
  return ret

In [74]:
x = np.array([[2], ['S']], dtype='O').T
ConditionalProbability(X, y, x, 1)

0.037037037037037035

### 计算后验概率（极大似然估计）
公式：
$$
P(Y=c_k)\prod_{j=1}^nP(X^{(j)}=x^{(j)} \mid Y=c_k),\ \ \ k=1,2,\cdots,K
$$

In [75]:
def PosteriorProbability(X, y, x):
  # X - 训练数据集的特征
  # y - 训练数据集的标签值
  # x - 待分类数据的特征
  # y_hat - 待分类数据的标签值
  label = np.unique(y)
  posterior = []
  for i in range(len(label)):
    y_hat = label[i]
    tmp = PriorProbability(y, y_hat) * ConditionalProbability(X, y, x, y_hat)
    posterior.append(tmp)
  return posterior

In [80]:
x = np.array([[2], ['S']], dtype='O').T
np.unique(y), PosteriorProbability(X, y, x)

(array([-1, 1], dtype=object), [0.06666666666666667, 0.02222222222222222])

### 确定实例x的类（极大似然估计）
公式：
$$
y=arg\max_{c_k}P(Y=c_k)\prod_{j=1}^nP(X^{(j)}=x^{(j)} \mid Y=c_k)
$$

In [81]:
def Determination(X, y, x):
  posterior = PosteriorProbability(X, y, x)
  label = np.unique(y)
  index = np.argmax(posterior)
  return label[index]

In [82]:
x = np.array([[2], ['S']], dtype='O').T
y_hat = Determination(X, y, x)

print("{}的类别为y={}".format(x[0], y_hat))

[2 'S']的类别为y=-1


## 贝叶斯估计

### 计算先验和条件概率（贝叶斯估计）

先验概率$P(Y=c_k)$的极大似然估计：
$$
P(Y=c_k)=\frac{\sum_{i=1}^NI(y_i=c_k)+\lambda}{N+K\lambda}，k=1,2,\dots,K
$$

+ $\lambda \ge 0$；当$\lambda = 0$时就是极大似然估计；$\lambda$常取1，这时称作拉普拉斯平滑。
+ 分子的$\lambda$保证分子不为0。
+ 分母的$K\lambda$保证$\sum_{k=1}^K P_\lambda(Y=c_k)=1$。

***

条件概率$P(X^{(j)}=a_{jl}|Y=c_k)$的极大似然估计：
$$
P(X^{(j)}=a_{jl}|Y=c_k)=\frac{\sum_{i=1}^NI(x_i^{(j)}=a_{jl},y_i=c_k)+\lambda}{\sum_{i=1}^NI(y_i=c_k)+S_j\lambda} \\ 
j=1,2,\dots,n；l=1,2,\dots,S_j；k=1,2,\dots,K
$$

其中：$x_i^{(j)}$是第$i$个样本的第$j$个特征；$a_{jl}$是第$j$个特征可能取得第$l$个值

+ 分子的$\lambda$保证分子不为0。
+ 分母的$S_j\lambda$保证$\sum_{l=1}^{S_j} P_\lambda(X^{(j)}=a_{jl} \mid Y=c_k)=1$。

In [83]:
def PriorProbabilityBayes(y, y_hat, lambd):
  prior = 0
  for i in range(y.shape[0]):
    prior += sign(y[i], y_hat)
  prior += lambd
  prior = prior / (y.shape[0] + len(np.unique(y)) * lambd)

  return prior

In [84]:
def ConditionalProbabilityBayes(X, y, x, y_hat, lambd):
  # X - 训练数据集的特征
  # y - 训练数据集的标签值
  # x - 待分类数据的特征
  # y_hat - 待分类数据的标签值
  
  ret = 1.0
  for j in range(x.shape[1]):
    numerator, denominator = 0, 0

    for i in range(X.shape[0]):
      denominator += sign(y[i], y_hat)
      numerator += (int)(sign(y[i], y_hat) and sign(X[i,j], x[0,j]))
    numerator += lambd
    denominator += len(np.unique(X[:, j])) * lambd
    ret *= numerator / denominator
  return ret

In [85]:
x = np.array([[2], ['S']], dtype='O').T
ConditionalProbabilityBayes(X, y, x, 1, 1)

0.05555555555555555

### 计算后验概率（贝叶斯估计）
公式：
$$
P(Y=c_k)\prod_{j=1}^nP(X^{(j)}=x^{(j)} \mid Y=c_k),\ \ \ k=1,2,\cdots,K
$$

In [86]:
def PosteriorProbabilityBayes(X, y, x, lambd):
  # X - 训练数据集的特征
  # y - 训练数据集的标签值
  # x - 待分类数据的特征
  # y_hat - 待分类数据的标签值
  label = np.unique(y)
  posterior = []
  for i in range(len(label)):
    y_hat = label[i]
    tmp = PriorProbabilityBayes(y, y_hat, lambd) * ConditionalProbabilityBayes(X, y, x, y_hat, lambd)
    posterior.append(tmp)
  return posterior

In [87]:
x = np.array([[2], ['S']], dtype='O').T
np.unique(y), PosteriorProbabilityBayes(X, y, x, 1)

(array([-1, 1], dtype=object), [0.061002178649237467, 0.032679738562091505])

### 确定实例x的类（极大似然估计）
公式：
$$
y=arg\max_{c_k}P(Y=c_k)\prod_{j=1}^nP(X^{(j)}=x^{(j)} \mid Y=c_k)
$$

In [88]:
def DeterminationBayes(X, y, x, lambd):
  posterior = PosteriorProbabilityBayes(X, y, x, lambd)
  label = np.unique(y)
  index = np.argmax(posterior)
  return label[index]

In [91]:
x = np.array([[2], ['S']], dtype='O').T
lambd = 1

y_hat_bayes = DeterminationBayes(X, y, x, lambd)

print("{}的类别为y={}".format(x[0], y_hat_bayes))

[2 'S']的类别为y=-1
